In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
pd.set_option("display.max_columns",75)

In [2]:
## FEATURE FUNCS

def strip_and_make_0_float(x):

    return x.str.split(expand=True)[0].astype(float)

def featurize(table):

    table.columns = [
        'rk','team','conf','g','rec','adjoe','adjde','barthag','efg_pct','efgd_pct',
        'tor','tord','orb','drb','ftr','ftrd','2p_pct','2pd_pct','3p_pct','3pd_pct','adj_t','wab'
    ]

    table = table[table['rk']!='Rk'].copy()
    table[['w','l']]=table['rec'].str.split(expand=True)[0].str.split("-",expand=True).astype(int)
    table['win_perc'] = table['w']/(table['w']+table['l'])

    table[['team','tourney']]=table['team'].str.split(",",expand=True)
    table['team'] = table['team'].str.replace(" seed","").apply(
        lambda x: ''.join([i for i in x if (i.isalpha())|(i==" ")])
    ).str.strip()


    table['adjoe'] = strip_and_make_0_float(table['adjoe'])
    table['adjde'] = strip_and_make_0_float(table['adjde'])
    table['barthag'] = strip_and_make_0_float(table['barthag'])
    table['efg_pct'] = strip_and_make_0_float(table['efg_pct'])
    table['efgd_pct'] = strip_and_make_0_float(table['efgd_pct'])
    table['tor'] = strip_and_make_0_float(table['tor'])
    table['tord'] = strip_and_make_0_float(table['tord'])
    table['orb'] = strip_and_make_0_float(table['orb'])
    table['drb'] = strip_and_make_0_float(table['drb'])
    table['ftr'] = strip_and_make_0_float(table['ftr'])
    table['ftrd'] = strip_and_make_0_float(table['ftrd'])
    table['2p_pct'] = strip_and_make_0_float(table['2p_pct'])
    table['2pd_pct'] = strip_and_make_0_float(table['2pd_pct'])
    table['3p_pct'] = strip_and_make_0_float(table['3p_pct'])
    table['3pd_pct'] = strip_and_make_0_float(table['3pd_pct'])
    table['adj_t'] = strip_and_make_0_float(table['adj_t'])
    table['wab'] = strip_and_make_0_float(table['wab'])

    table['tourney'] = table['tourney'].fillna('no tourney').str.strip()

    conferences = ['WCC', 'Amer', 'B12', 'ACC', 'SEC', 'BE', 'P12', 'B10', 'MWC',
        'MVC', 'A10', 'OVC', 'CUSA', 'AE', 'SC', 'WAC', 'Sum', 'CAA',
        'MAAC', 'MAC', 'Ivy', 'ASun', 'Pat', 'SB', 'BW', 'BSth', 'BSky',
        'NEC', 'Horz', 'SWAC', 'MEAC', 'Slnd']

    tourney_values = {
        'no tourney':128,
        'R68':68,
        'R64':64,
        'R32':32,
        'Sweet Sixteen':16,
        'Elite Eight':8,
        'Final Four':4,
        'Finals':2,
        'CHAMPS':1,
    }

    table['OUTCOME'] = table['tourney'].map(tourney_values)

    for c in conferences:
        table[c] = np.where(table['conf']==c,1,0)

    return table

In [3]:
years = [
    2008,
    2009,
    2010,
    2011,
    2012,
    2013,
    2014,
    2015,
    2016,
    2017,
    2018,
    2019,
    2021,
    2022
         ]

out = pd.DataFrame()
for year in years:
    print(year)

    url = f"https://barttorvik.com/trank.php?year={year}"

    tables = pd.read_html(url)

    table = tables[0]

    tmp = featurize(table)

    tmp['year']=year

    out = pd.concat([out,tmp])

2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2021
2022


In [4]:
out

,rk,team,conf,g,rec,adjoe,adjde,barthag,efg_pct,efgd_pct,tor,tord,orb,drb,ftr,ftrd,2p_pct,2pd_pct,3p_pct,3pd_pct,adj_t,wab,w,l,win_perc,tourney,OUTCOME,WCC,Amer,B12,ACC,SEC,BE,P12,B10,MWC,MVC,A10,OVC,CUSA,AE,SC,WAC,Sum,CAA,MAAC,MAC,Ivy,ASun,Pat,SB,BW,BSth,BSky,NEC,Horz,SWAC,MEAC,Slnd,year
0,1,Kansas,B12,39,37-3 13–3,121.4,85.5,0.9825,56.6,44.3,19.1,22.4,37.6,29.1,36.4,31.0,55.4,41.2,39.7,32.8,68.6,9.9,37,3,0.925000,CHAMPS,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008
1,2,Memphis,CUSA,40,38-2 16–0,117.0,86.1,0.9715,52.7,43.4,16.6,22.5,37.8,29.6,40.4,31.7,52.9,42.6,34.9,30.2,70.2,8.9,38,2,0.950000,Finals,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008
2,3,UCLA,P10,38,35-4 16–2,116.2,86.8,0.9664,52.2,46.5,19.1,21.8,39.2,27.2,37.3,25.6,52.4,45.4,34.4,32.9,66.1,10.8,35,4,0.897436,Final Four,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008
3,4,North Carolina,ACC,39,36-3 14–2,122.9,91.9,0.9659,53.0,48.2,18.8,21.1,42.5,29.0,38.0,25.7,52.1,47.8,37.2,32.6,74.7,11.9,36,3,0.923077,Final Four,4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008
4,5,Wisconsin,B10,36,31-5 16–2,112.6,85.4,0.9600,50.6,43.4,19.5,21.9,36.2,28.6,39.9,25.4,49.3,41.7,35.6,31.3,63.1,8.3,31,5,0.861111,Sweet Sixteen,16,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,354,Columbia,Ivy,25,4-22 1–13,91.5,114.2,0.0725,48.5,54.8,21.5,16.4,27.5,28.8,26.1,25.1,46.4,53.4,34.9,37.8,69.6,-18.1,4,22,0.153846,no tourney,128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2022
367,355,Central Connecticut,NEC,32,8-24 5–13,90.0,112.5,0.0712,46.0,52.9,20.6,18.8,26.3,31.8,29.1,28.8,42.6,53.3,34.2,34.8,65.1,-19.0,8,24,0.250000,no tourney,128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2022
368,356,Delaware St,MEAC,25,2-26 0–14,86.5,110.1,0.0587,45.0,51.1,25.6,19.3,27.0,34.7,32.4,34.2,43.4,48.6,32.2,36.1,68.1,-22.8,2,26,0.071429,no tourney,128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2022
369,357,Eastern Illinois,OVC,29,5-26 3–15,83.0,108.4,0.0442,44.2,53.4,24.1,18.2,20.6,32.6,31.2,24.2,42.3,55.8,31.8,33.5,64.9,-20.3,5,26,0.161290,no tourney,128,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022


In [5]:
training_mask = out['year'].isin([2008,
    2009,
    2010,
    2011,
    2012,
    2013,
    2014,
    2015,
    2016,
    2017,
    2018,
    2019,
    2021,])
validation_mask = out['year'].isin([2022])


In [27]:
features = ['adjoe', 'adjde', 'barthag',
       'efg_pct', 'efgd_pct', 'tor', 'tord', 'orb', 'drb', 'ftr', 'ftrd',
       '2p_pct', '2pd_pct', '3p_pct', '3pd_pct',
       'win_perc', 
       'WCC', 'Amer', 'B12', 'ACC', 'SEC',
       'BE', 'P12', 'B10', 'MWC', 'MVC', 'A10', 'OVC', 'CUSA', 'AE', 'SC',
       'WAC', 'Sum', 'CAA', 'MAAC', 'MAC', 'Ivy', 'ASun', 'Pat', 'SB', 'BW',
       'BSth', 'BSky', 'NEC', 'Horz', 'SWAC', 'MEAC', 'Slnd'
       ]
target = 'OUTCOME'

In [28]:
X = out[training_mask][features].copy()
y = out[training_mask][target].copy()

In [29]:
model = Pipeline(
    steps=[
    ('scaler',MinMaxScaler()),
    ('learner',RandomForestRegressor(n_estimators=50,random_state=50,max_depth=8))
    ]
)

In [30]:
model.fit(X,y)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('learner',
                 RandomForestRegressor(max_depth=8, n_estimators=50,
                                       random_state=50))])

In [31]:
yfit = model.predict(X)

In [32]:
Xx = out[validation_mask][features].copy()
yy = out[validation_mask][target].copy()
yval = model.predict(Xx)

In [33]:
val_result = pd.DataFrame({
    'team':out[validation_mask]['team'],
    'OUTCOME':yy})
val_result['PREDICTION_NUMERIC'] = yval
val_result['PREDICTION_RANK']=val_result['PREDICTION_NUMERIC'].rank(ascending=True,)

In [34]:
val_result.sort_values(by='PREDICTION_RANK',ascending=True).head(50)

,team,OUTCOME,PREDICTION_NUMERIC,PREDICTION_RANK
0,Gonzaga,16,9.138248,1.0
2,Kansas,1,10.689049,2.0
4,Baylor,32,12.832259,3.0
1,Houston,8,13.373869,4.0
5,Duke,4,13.878481,5.0
7,Villanova,4,15.217023,6.0
6,Tennessee,32,15.690148,7.0
10,Kentucky,64,17.065737,8.0
13,Auburn,32,18.536197,9.0
9,UCLA,16,19.031298,10.0
